In [1]:
import sys, os

import pickle
import csv
from scipy import sparse, stats
import gzip
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from stereosite.plot import scii_circos, scii_net
from stereosite.scii import intensities_count
from pycirclize.utils import calc_group_spaces, ColorCycler
from pycirclize import Circos
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.colors import ListedColormap





/home/zhaoxuelin/.conda/envs/cell2loc_env/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [2]:
def _sankey_2d(data:pd.DataFrame, cmap="tab20", left_labels:list=None, right_labels:list=None,
               l_width:float = -1.3, r_width:float = 2,
              aspect:float=4, fontsize=4, save=None, close_plot=False, patch_alpha:float=0.99, link_alpha:float=0.65,
              interval:float=0.0, strip_color='left', ax:mpl.axes.Axes=None, dpi:float=300
              ):
    '''
    Make Sankey Diagram
    
    Inputs:
        data: pandas.DataFrame.
            contains columns left, right, mid_l, mid_r, left_weight, right_weight, mid_l_weight, mid_r_weight
        cmap: str|dict.
            Define colors of each patch. User can set matplotlib's colormap (e.g. viridis, jet, tab10) or label_name -> color dict (e.g. dict(A="red", B="blue", C="green", ...)).
        left_labels: list[str] | array[str].
            order of the left labels in the diagram
        right_labels: list[str] | array[str].
            order of the right labels in the diagram
        aspect: float.
            vertical extent of the diagram in units of horizontal extent
        
    Output:
        None
    '''  
    if ax==None:
        fig = plt.figure(dpi=dpi)
        ax = fig.add_subplot(1, 1, 1)

    if len(data[(data.left.isnull()) | (data.right.isnull())]):
        raise NullsInFrame('Sankey graph dose not support null values.')

    #Identify all labels that appear 'left' or 'right'
    allLabels = pd.Series(np.r_[data.left.unique(), data.right.unique()]).unique()

    #Identify left labels
    if left_labels == None:
        left_labels = data.left.unique()[::-1]
    else:
        _check_data_matches_labels(left_labels, data['left'])
    if right_labels == None:
        right_labels = data.right.unique()[::-1]
    else:
        _check_data_matches_labels(right_labels, data['right'])

    if isinstance(cmap, str):
        color_dict = {}
        colorPalette = sns.color_palette(cmap, len(allLabels))
        for i, label in enumerate(allLabels):
            color_dict[label] = colorPalette[i]
    elif isinstance(cmap, dict):
        color_dict = cmap
    else:
        raise Exception("cmap must be string representing the matplotlib's colormap or dict")

    #Determine widths of individual strips
    from collections import defaultdict
    ns_l = defaultdict()
    ns_r = defaultdict()
    for leftLabel in left_labels:
        leftDict = {}
        rightDict = {}
        for rightLabel in right_labels:
            leftDict[rightLabel] = data[(data.left == leftLabel) & (data.right == rightLabel)].left_weight.sum()
            rightDict[rightLabel] = data[(data.left == leftLabel) & (data.right == rightLabel)].right_weight.sum()
        ns_l[leftLabel] = leftDict
        ns_r[leftLabel] = rightDict

    # Determine positions of left label patches and total widths
    leftWidths = defaultdict()
    for i, leftLabel in enumerate(left_labels):
        myD = {}
        myD['left'] = data[data.left == leftLabel].left_weight.sum()
        if i == 0:
            myD['bottom'] = 0
            myD['top'] = myD['left']
        else:
            myD['bottom'] = leftWidths[left_labels[i - 1]]['top'] + interval*data.left_weight.sum()
            myD['top'] = myD['bottom'] + myD['left']
            topEdge = myD['top']
        leftWidths[leftLabel] = myD

    # Determine positions of right label patches and total widths
    rightWidths = defaultdict()
    for i, rightLabel in enumerate(right_labels):
        myD = {}
        myD['right'] = data[data.right == rightLabel].right_weight.sum()
        if i == 0:
            myD['bottom'] = 0
            myD['top'] = myD['right']
        else:
            myD['bottom'] = rightWidths[right_labels[i-1]]['top'] + interval*data.right_weight.sum()
            myD['top'] = myD['bottom'] + myD['right']
            topEdge = myD['top']
        rightWidths[rightLabel] = myD

    #Total vertical extent of diagram
    # l_width = -0.3
    # r_width = 1.3
    xMax = topEdge/aspect

    #Draw vertical bars on left and right of each label's section & print label
    for leftLabel in left_labels:
        ax.fill_between(
            [l_width*xMax, 0],
            2 * [leftWidths[leftLabel]['bottom']],
            2 * [leftWidths[leftLabel]['bottom'] + leftWidths[leftLabel]['left']],
            color = color_dict[leftLabel],
            alpha=patch_alpha,
            edgecolor='k',
            linewidth=0.3,
        )
        ax.text(
            l_width/2 * xMax,
            leftWidths[leftLabel]['bottom'] + 0.5*leftWidths[leftLabel]['left'],
            leftLabel,
            {'ha': 'center', 'va': 'center'},
            fontsize=fontsize
        )
    for rightLabel in right_labels:
        ax.fill_between(
            [xMax, r_width*xMax], 2 * [rightWidths[rightLabel]['bottom']],
            2 * [rightWidths[rightLabel]['bottom'] + rightWidths[rightLabel]['right']],
            color = color_dict[rightLabel],
            alpha=patch_alpha,
            edgecolor='k',
            linewidth=0.3,
        )
        ax.text(
            (r_width+l_width/2) * xMax,
            rightWidths[rightLabel]['bottom'] + 0.5*rightWidths[rightLabel]['right'],
            rightLabel,
            {'ha': 'center', 'va': 'center'},
            fontsize = fontsize
        )

    # Plot strips
    for leftLabel in left_labels:
        for rightLabel in right_labels:
            if strip_color == 'left':
                labelColor = leftLabel
            else:
                labelColor = rightLabel
            if len(data[(data.left == leftLabel) & (data.right == rightLabel)]) > 0 :
                # Create array of y values for each strip, half at let value,
                # half at right
                ys_d = np.array(50 * [leftWidths[leftLabel]['bottom']] + 50 * [rightWidths[rightLabel]['bottom']])
                ys_d = np.convolve(ys_d, 0.05*np.ones(20), mode='valid')
                ys_d = np.convolve(ys_d, 0.05*np.ones(20), mode='valid')
                ys_u = np.array(50 * [leftWidths[leftLabel]['bottom'] + ns_l[leftLabel][rightLabel]] + 50 * [rightWidths[rightLabel]['bottom'] + ns_r[leftLabel][rightLabel]])
                ys_u = np.convolve(ys_u, 0.05 * np.ones(20), mode='valid')
                ys_u = np.convolve(ys_u, 0.05 * np.ones(20), mode='valid')

                # Update bottom edges at each label so next strip starts at the right place
                leftWidths[leftLabel]['bottom'] += ns_l[leftLabel][rightLabel]
                rightWidths[rightLabel]['bottom'] += ns_r[leftLabel][rightLabel]
                ax.fill_between(
                    np.linspace(0, xMax, len(ys_d)), ys_d, ys_u, alpha=link_alpha,
                    color = color_dict[labelColor]
                )
    ax.axis('off')
    if save != None:
        plt.savefig(save, bbox_inches='tight', dpi=dpi)
    if close_plot:
        plt.close()
        
def pattern_decomposition(filter_matrix, lr_pathway_dict, cells, components=5, W_filter=0.5, H_filter=0.2):
    
    alllr = set(zip(cellchatDB['ligand'] , cellchatDB['receptor']))
    filter_matrix = filter_matrix[filter_matrix.index.map(lambda x: x in alllr)]

    sender_df = pd.DataFrame(index = [lr_pathway_dict[f'{x[0]}_{x[1]}'] if f'{x[0]}_{x[1]}' in lr_pathway_dict.keys() else cellchatDB[(cellchatDB['ligand'] == x[0]) & \
                                                            (cellchatDB['receptor'] == x[1])]['pathway_name'].values[0] for x in filter_matrix.index])
    for cell in cells:
        sender_df[cell] = filter_matrix[cell].sum(axis=1).values
    sender_df = sender_df.groupby(sender_df.index).sum()
    from sklearn.decomposition import NMF
    model = NMF(n_components = components, init='random', random_state=0)
    W = model.fit_transform(sender_df.T.values)
    H = model.components_
    
    normalized_W = (W - W.min(axis=1, keepdims=True)) / (W.max(axis=1, keepdims=True) - W.min(axis=1, keepdims=True))
    normalized_W[normalized_W < W_filter] = 0

    df_W = pd.DataFrame(normalized_W, index = sender_df.T.index)
    left = []
    right = []
    left_weight = []
    right_weight = []
    for cell, row in df_W.iterrows():
        for i, v in row.items():
            if v == 0:
                continue
            left.append(cell)
            right.append(f"Pattern {i+1}")
            left_weight.append(v)
            right_weight.append(v)
    sankey_df_W = pd.DataFrame({'left': left, 'right': right, 'left_weight': left_weight, 'right_weight': right_weight}, index=range(len(left)))

    normalized_H = (H - H.min(axis=0, keepdims=True)) / (H.max(axis=1, keepdims=True) - H.min(axis=1, keepdims=True))
    normalized_H[normalized_H < H_filter] = 0
    df_H = pd.DataFrame(normalized_H, columns = sender_df.T.columns)
    left = []
    right = []
    left_weight = []
    right_weight = []
    average_v = normalized_H.sum()/(normalized_H.shape[0]*normalized_H.shape[1])
    for i, row in df_H.iterrows():
        for pathway, v in row.items():
            if v == 0:
                continue
            left.append(f"Pattern {i+1}")
            right.append(pathway)
            left_weight.append(v)
            right_weight.append(v)
    sankey_df_H = pd.DataFrame({'left': left, 'right': right, 'left_weight': left_weight, 'right_weight': right_weight}, index=range(len(left)))
    return sankey_df_W, sankey_df_H

### SCII

In [ ]:
adata = sc.read_h5ad("./cellanno/cell2location_map_cell/sp.h5ad")
interactiondb_file = "StereoSiTE/stereosite/datasets/LR_database/CellChatDB.human.csv"

In [ ]:
scii_dict = intensities_count(adata, interactiondb_file,
                              #distance_threshold = 50,
                              distance_threshold = {'Secreted Signaling': 200, 'ECM-Receptor': 200, 'Cell-Cell Contact': 30},
                              anno = 'cell2loc_anno')

In [ ]:
cells = adata.obs.cell2loc_anno.cat.categories.tolist()

columns = []
for sender in cells:
    for receiver in cells:
        if sender != receiver:
            columns.append((sender, receiver))

gene_list = []
plot_data = {'intensities': scii_dict['intensities'].loc[:,columns],
            'pvalues': scii_dict['pvalues'].loc[:,columns]}
plot_data['intensities'] = plot_data['intensities'][[all([all([x not in gene_list for x in y.split("_")]) for y in z]) for z in plot_data['intensities'].index]]
plot_data['pvalues'] = plot_data['pvalues'][[all([all([x not in gene_list for x in y.split("_")]) for y in z]) for z in plot_data['pvalues'].index]]

filter_matrix = plot_data['intensities'][(plot_data['pvalues'] < 0.05) & (plot_data['intensities']>3000) & (plot_data['intensities'])].dropna(axis=0, how='all').fillna(0)

In [ ]:
interactiondb_file = "./StereoSiTE/stereosite/datasets/LR_database/CellChatDB.human.csv"
cellchatDB = pd.read_csv(interactiondb_file)

lr_pathway_dict = dict(zip(cellchatDB['interaction_name'].values, cellchatDB['pathway_name'].values))

In [ ]:
filter_matrix = plot_data['intensities'][(plot_data['pvalues'] < 0.05) & (plot_data['intensities']>3000) & (plot_data['intensities'])].dropna(axis=0, how='all').fillna(0)
print(filter_matrix.shape)
anno = 'cell2loc_anno'
cells = adata.obs[anno].cat.categories.to_list()
columns = []
for sender in cells:
    for receiver in cells:
        if sender != receiver:
            columns.append((sender, receiver))
interaction_matrix = filter_matrix[columns].T
interaction_matrix.index = [f'{x[0]}_{x[1]}' for x in interaction_matrix.index]
interaction_matrix.columns = [f'{x[0]}-{x[1]}' for x in interaction_matrix.columns]

### Outgoing communication patterns sankey plot

In [ ]:
components=4
anno = 'cell2loc_anno'

sankey_df_W, sankey_df_H = pattern_decomposition(filter_matrix, lr_pathway_dict, cells,components=components,W_filter=0.1,H_filter=0.1)

#colors_dict = dict(zip(adata.obs[anno].cat.categories, adata.uns[f'{anno}_colors']))
colors_dict = cellcolors
colors_dict = {k.replace(" cells",""):v for k,v in colors_dict.items()}
color_list = plt.get_cmap('Set2', components).colors
for i in range(components):
    colors_dict[f"Pattern {i+1}"] = color_list[i] #"#BFC1C2"
    
from stereosite.plot import sankey
sankey_df_W = sankey_df_W.dropna()
sankey_df_W["left"] = sankey_df_W["left"].map(lambda x:x.replace(" cells",""))

#sankey.sankey_2d(sankey_df_W.sort_values(by=['right']), cmap=colors_dict, fontsize=8, strip_color='right')
fig,axs = plt.subplots(1,1,figsize=(5,5))
_sankey_2d(sankey_df_W.sort_values(by=['right']), cmap=colors_dict,ax=axs, fontsize=10, strip_color='right')

In [ ]:
color_list_1 = plt.get_cmap('tab20', sankey_df_H['right'].nunique()).colors
i = 0
for pathway_name in sankey_df_H['right'].unique():
    colors_dict[pathway_name] = color_list_1[i]
    i+=1
    
fig,axs = plt.subplots(1,1,figsize=(5,5))
_sankey_2d(sankey_df_H, cmap=colors_dict,ax=axs, fontsize=10)

 ### Incoming communication patterns sankey plot
 receiver decompositio

In [ ]:
filter_matrix_1 = filter_matrix.copy()
filter_matrix_1.columns = filter_matrix_1.columns.swaplevel(i=0, j=1)
sankey_df_W_r, sankey_df_H_r = pattern_decomposition(filter_matrix_1, lr_pathway_dict, cells,components=components, W_filter=0.1, H_filter=0.1)

sankey_df_W_r = sankey_df_W_r.dropna()
import matplotlib.pyplot as plt
anno = 'cell2loc_anno'
colors_dict = cellcolors
colors_dict = {k.replace(" cells",""):v for k,v in colors_dict.items()}
color_list = plt.get_cmap('Set2', components).colors
for i in range(components):
    colors_dict[f"Pattern {i+1}"] = color_list[i] #"#BFC1C2"

from stereosite.plot import sankey
sankey_df_W_r["left"] = sankey_df_W_r["left"].map(lambda x:x.replace(" cells",""))
fig,axs = plt.subplots(1,1,figsize=(5,5))
_sankey_2d(sankey_df_W_r.sort_values(by=['right']), cmap=colors_dict,ax=axs, fontsize=10, strip_color='right')

In [ ]:
pathway_names = list(set(sankey_df_H['right'].unique()) | set(sankey_df_H_r['right'].unique()))
color_list_1 = plt.get_cmap('Set3', len(pathway_names)).colors
i = 0
for pathway_name in pathway_names:
    colors_dict[pathway_name] = color_list_1[i]
    i+=1
#sankey.sankey_2d(sankey_df_H_r, cmap=colors_dict, fontsize=8)
fig,axs = plt.subplots(1,1,figsize=(5,5))
_sankey_2d(sankey_df_H_r, cmap=colors_dict,ax=axs, fontsize=10)

### Interaction

In [ ]:
def _cell_pairs_generate(cells:list, separator:str="_"):
    cell_pairs = {}
    for c1 in cells:
        for c2 in cells:
            cell_pairs[f"{c1}{separator}{c2}"] = (c1, c2)
    return cell_pairs

def scii_interaction_matrix_process(interaction_matrix:pd.DataFrame,
                                    cells_sorted:list,
                                    cell_lr_separator:str="|",
                                    ) -> tuple:
    '''
    Input
        interaction_matrix: DataFrame. Matrix contains interesting interaction, index represents cell cell pairs
                            while column represents ligand receptor pairs
        cells: List of cell type names, which will be used to separate cells of the cell pair.
        separator: Separator used to combine ligand with receptor genes into LR pairs.
        cell_lr_separator: Separator used to combine cells with LR genes.
    Return
        (sectors:dict, links:list, genes:set)
        sectors: The dictionary containing all vertices and their weight. {sender-ligand: value, receiver-receptor: value, ...}
        links: List containing all links between sectors. [[sender-ligand, receiver-receptor, value], ...]
        genes: Set contains names of all genes.
    '''

    #Normalize the value into 0~100
    interaction_matrix = interaction_matrix.fillna(0).T

    scii_min, scii_max = interaction_matrix.min().min(), interaction_matrix.max().max()
    norm_mt = ((interaction_matrix - scii_min)/(scii_max - scii_min)*100).apply(round).astype(int)

    #Generate sectors and cell groups
    sectors1 = defaultdict()
    sectors2 = defaultdict()
    genes = set()
    cells = set()
    links = []
    for index, row in norm_mt.iterrows():
        sender, receiver = index
        cells.add(sender)
        cells.add(receiver)
        for LR, value in row.items():
            if value == 0:
                continue
            ligand, receptor = LR
            v1 = f"{sender}{cell_lr_separator}{ligand}"
            v2 = f"{receiver}{cell_lr_separator}{receptor}"
            if sender not in sectors1.keys():
                sectors1[sender] = defaultdict(int)
            if receiver not in sectors2.keys():
                sectors2[receiver] = defaultdict(int)
            sectors1[sender][ligand] += value
            sectors2[receiver][receptor] += value
            genes.add(ligand)
            genes.add(receptor)
            links.append([v1, v2, value])

    sectors = defaultdict(int)
    cells = [x for x in cells_sorted if x in cells]
    for cell in cells:
        if cell in sectors1.keys():
            for ligand, value in sectors1[cell].items():
                sectors[f"{cell}{cell_lr_separator}{ligand}"] += value
        if cell in sectors2.keys():
            for receptor, value in sectors2[cell].items():
                sectors[f"{cell}{cell_lr_separator}{receptor}"] += value
    return sectors, links, genes, list(cells)

def _cells_lr_circos(interaction_matrix:pd.DataFrame,
                  cells:list=None,
                  cell_colors='Set3',
                  gene_colors='tab20',
                  link_color_palette:str='tab20',
                  separater:str="-",
                  cell_lr_separator:str="|",
                  label_size:float=8,
                  dpi:float=300,
                  scii_tensor:bool=False,
                  islegend:bool = True,
                  save:str=None,
                  ):
    '''
    Input:
        interaction_matrix: pandas.DataFrame.
            The dataframe that contains the spatial cell interaction intensity(SCII) values of each interaction.
            The index represents cell_cell pairs and the column represents ligand_receptor pairs.
        cells: list.
            The list contains the names of all cell types. This will be used to separate the sender and receiver cells that were 
            combined to create index names for the interaction_matrix.
        cell_colors: str|dict.
            Define colors of difference cell type. User can set matplotlib's colormap (e.g. viridis, jet, tab10) or label_name -> color dict (e.g. dict(A="red", B="blue", C="green", ...)).
        gene_colors: str|dict.
            Define colors of difference gene. User can set matplotlib's colormap (e.g. viridis, jet, tab10) or label_name -> color dict (e.g. dict(A="red", B="blue", C="green", ...)).
        link_color_palette: str|list.
            Define colors of difference link. User can set matplotlib's colormap (e.g. viridis, jet, tab10) or list contain colors.
        separator: str.
            Separator used to combine ligand with receptor genes into LR pairs.
        cell_lr_separator: str.
            Separator used to combine cells with LR genes.
        scii_tensor: bool.
            If the interaction_matrix was generated by scii, set the scii_tensor=True.
        save: str.
            File name of the figure that will be saved.
    Return:    
        None
    '''

    if scii_tensor:
        sectors, links, genes = scii_tensor_interaction_matrix_process(interaction_matrix, cells=cells, separator=separater, cell_lr_separator=cell_lr_separator)
    else:
        sectors, links, genes, cells = scii_interaction_matrix_process(interaction_matrix, cells,cell_lr_separator=cell_lr_separator)
        

    cell_groups = defaultdict(list)
    for key in sectors.keys():
        cell, gene = key.split(cell_lr_separator, 1)
        cell_groups[cell].append(key)
    group_sizes = [len(value) for key, value in cell_groups.items()]

    #Generate the color palette for cells, genes and links
    #cmap1 = plt.get_cmap('tab20b')
    #cmap2 = plt.get_cmap('tab20c')
    
    if isinstance(cell_colors, str):
        cell_color_palette = plt.get_cmap(cell_colors, len(cells)).colors
        cell_colors = dict(zip(cells, cell_color_palette[0:len(cells)]))
    elif isinstance(cell_colors, dict):
        cell_colors = cell_colors
    else:
        raise Exception("cell_colors must be string representing the matplotlib's colormap or dict")

    if isinstance(gene_colors, str):
        gene_color_palette = plt.get_cmap(gene_colors, len(genes)).colors
        gene_colors = dict(zip(sorted(list(genes)), gene_color_palette[0:len(genes)]))
    elif isinstance(gene_colors, dict):
        gene_colors = gene_colors
    else:
        raise Exception("gene_colors must be string representing the matplotlib's colormap or dict")


    lr_links = sorted(list(set([(x[0].split(cell_lr_separator, 1)[1], x[1].split(cell_lr_separator, 1)[1]) for x in links])))
    if isinstance(link_color_palette, str):
        link_color_palette = plt.get_cmap(link_color_palette, len(lr_links)).colors
    elif isinstance(link_color_palette, list):
        if len(link_color_palette) < len(lr_links):
            raise Exception("the length of link_color_palette less than then number of lr_links: {0} < {1}".format(len(link_color_palette), len(lr_links)))
    link_colors = dict(zip(lr_links, link_color_palette[0:len(lr_links)]))

    spaces = calc_group_spaces(group_sizes, space_bw_group=10, space_in_group=1)
    circos = Circos(sectors, space=spaces)

    # Plot sector track
    #ColorCycler.set_cmap("Set3")
    for sector in circos.sectors:
        track = sector.add_track(r_lim=(90, 95))
        track.axis(fc=gene_colors[sector.name.split(cell_lr_separator, 1)[1]])
        #track.text(sector.name.split("-", 1)[1], fontsize=5, r=92, orientation="vertical")

    #ColorCycler.set_cmap("tab10")
    for cell, group in cell_groups.items():
        group_deg_lim = circos.get_group_sectors_deg_lim(group)
        circos.rect(r_lim=(100, 103), deg_lim=group_deg_lim, fc=cell_colors[cell], ec="black", lw=0.5)
        group_center_deg = sum(group_deg_lim)/2
        circos.text(cell, r=106, deg=group_center_deg, adjust_rotation=True, fontsize=label_size)

    #Plot links
    for sender, receiver, value in links:
        ligand, receptor = sender.split(cell_lr_separator, 1)[1], receiver.split(cell_lr_separator, 1)[1]
        circos.link((sender, sectors[sender]-value, sectors[sender]), (receiver, sectors[receiver]-value, sectors[receiver]), color=link_colors[(ligand, receptor)], direction=1)
        sectors[sender] -= value
        sectors[receiver] -= value
        
    fig = circos.plotfig()
    #Plot legend
    rect_handles = []
    for link, color in link_colors.items():
        rect_handles.append(Patch(color=color, label=f"{link[0]}-{link[1]}"))
    
    if islegend:
        rect_legend = circos.ax.legend(
            handles = rect_handles,
            bbox_to_anchor=(1.1, 1.0),
            fontsize=6,
            title="Ligand-Receptor",
        )
        circos.ax.add_artist(rect_legend)

        scatter_handles = []
        for gene, color in gene_colors.items():
            scatter_handles.append(Line2D([], [], color=color, marker="o", label=gene, ms=6, ls="None"))
        scatter_legend = circos.ax.legend(
            handles=scatter_handles,
            bbox_to_anchor=(1.5, 1.0),
            fontsize=6,
            title="Gene",
            handlelength=2,
        )
    if save != None:
        fig.savefig(save, dpi=dpi)

In [ ]:
cells_new = [x.replace(" cells","") for x in cells]
cells_new1 = ['B',
             'Endothelial',
             'Smooth muscle',
             'Epithelial',
             'T',
             'Fibroblasts',
             'Mast',
             'Myeloid',
             'Pericytes',
             'Plasma']

filter_matrix_new = filter_matrix.copy()
filter_matrix_new.columns = filter_matrix_new.columns.map(lambda x:(x[0].replace(" cells",""),x[1].replace(" cells","")))

In [ ]:
pathway_name = 'EGF'
pathway_indexs = list(set(cellchatDB[cellchatDB['pathway_name'] == pathway_name]['interaction_name'].str.replace("_", "-", 1).to_list()) & set(interaction_matrix.columns))
pathway_matrix = filter_matrix_new[filter_matrix_new.index.map(lambda x: f"{x[0]}-{x[1]}" in pathway_indexs)]
_cells_lr_circos(pathway_matrix, cells_new1, 
                            cell_colors = colors_dict
                           )

In [ ]:
filter_matrix_new = filter_matrix.copy()
filter_matrix_new.columns = [(x[0].replace(" cells",""),x[1].replace(" cells","")) for x in filter_matrix_new.columns]
_cells_lr_circos(filter_matrix_new,cells = cells_new1,cell_colors=colors_dict,
                 islegend=False,label_size=19,dpi=900,
                 save=f"./scii/scii_circos.tif")